In [29]:
import pickle
from graph import Graph
from part import Part
from typing import List, Set, Dict, Tuple
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset



In [13]:
with open('data/graphs.dat', 'rb') as file:
    all_graphs: List[Graph] = pickle.load(file)
    X_train, X_temp, y_train, y_temp = train_test_split(list(map(lambda g: g.get_parts(), all_graphs)), all_graphs, test_size=0.3, random_state=0)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)


print(X_train[0])
print(y_train[0])
print(len(y_train))
print(y_train[0].get_edges())

{Part(PartID=118, FamilyID=0), Part(PartID=127, FamilyID=48), Part(PartID=119, FamilyID=46), Part(PartID=38, FamilyID=3), Part(PartID=686, FamilyID=8), Part(PartID=83, FamilyID=2)}
7811
{Node(NodeID=0, Part=Part(PartID=118, FamilyID=0)): [Node(NodeID=2, Part=Part(PartID=119, FamilyID=46)), Node(NodeID=1, Part=Part(PartID=127, FamilyID=48)), Node(NodeID=3, Part=Part(PartID=38, FamilyID=3)), Node(NodeID=5, Part=Part(PartID=686, FamilyID=8)), Node(NodeID=4, Part=Part(PartID=83, FamilyID=2))], Node(NodeID=1, Part=Part(PartID=127, FamilyID=48)): [Node(NodeID=0, Part=Part(PartID=118, FamilyID=0))], Node(NodeID=2, Part=Part(PartID=119, FamilyID=46)): [Node(NodeID=0, Part=Part(PartID=118, FamilyID=0))], Node(NodeID=3, Part=Part(PartID=38, FamilyID=3)): [Node(NodeID=0, Part=Part(PartID=118, FamilyID=0))], Node(NodeID=4, Part=Part(PartID=83, FamilyID=2)): [Node(NodeID=0, Part=Part(PartID=118, FamilyID=0))], Node(NodeID=5, Part=Part(PartID=686, FamilyID=8)): [Node(NodeID=0, Part=Part(PartID=118, 

In [31]:
# Prepare the data for embedding:
# 1. Map parts to indices which will determine one-hot encoding representation
# 2. Create a training set for the embedding. Each training will consist of one hot encoding of a part and the one hot encoding of one of its neighbors

# returns a dictionary of all parts in the training set and the size of the dictionary
# the value determines one-hot encoding
def map_parts_to_index(X_train: List[Set[Part]]) -> (Dict[Part, int], int):
    parts_list = [part for parts in X_train for part in parts]
    parts_dict = dict.fromkeys(parts_list)
    counter = 0
    for i in parts_dict:
        parts_dict[i] = counter
        counter += 1
    return parts_dict, len(parts_dict)

# returns a list of tuples. Each tuple contains a one-hot encoding of a part and a one-hot encoding of one of its neighbors
def create_embedding_training_set(X_train: List[Set[Part]], graphs: List[Graph]) -> List[Tuple[List[int], List[int]]]:
    training_set = []
    mapped_parts = map_parts_to_index(X_train)[0]
    mapped_parts_size = map_parts_to_index(X_train)[1]
    base_vector = [0] * mapped_parts_size
    # iterate through each graph in the training set
    for graph in graphs:
        # iterate through all edges in a graph. One part is the key, its neighbors are the values
        for node in graph.get_edges():
            part = node.get_part()
            part_one_hot_encoded = base_vector.copy()
            part_one_hot_encoded[mapped_parts[part]] = 1
            # iterate through all neighbors af a part. 
            for neighbor_node in graph.get_edges()[node]:
                neighbor_part = neighbor_node.get_part()
                neighbor_part_one_hot_encoded = base_vector.copy()
                neighbor_part_one_hot_encoded[mapped_parts[neighbor_part]] = 1
                training_set.append((part_one_hot_encoded, neighbor_part_one_hot_encoded))
            
    return training_set

print(len(create_embedding_training_set(X_train, y_train)))

# Create a dataset with the right format for the netowrk
class EmbeddingDataset(Dataset):
    def __init__(self, data):
        self.data = data  # list of tuples

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        part, neighbor = self.data[idx]
        return torch.tensor(part, dtype=torch.float), torch.tensor(neighbor, dtype=torch.float)
    
embedding_training_set = EmbeddingDataset(create_embedding_training_set(X_train, y_train))



103332


In [36]:
# Create neural network to learn the embeddings

# Train on an efficient device if possible
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


# Define the network architecture
class EmbeddingNetwork(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int):
        super(EmbeddingNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size) 
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        # Note: No softmax here when using nn.CrossEntropyLoss
        return out
    

# Define the model
input_size = map_parts_to_index(X_train)[1] # Number of different parts in the training set
hidden_size = 10 # Number of embedding dimension
output_size = map_parts_to_index(X_train)[1] # Number of different parts in the training set
model = EmbeddingNetwork(input_size, hidden_size, output_size)
model.to(device)

print(model)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss() # Cross-entropy loss
optimizer = torch.optim.Adam(model.parameters()) # Example optimizer

# Train the model
train_dataset = create_embedding_training_set(X_train, y_train)
train_loader = DataLoader(dataset=embedding_training_set, batch_size=64, shuffle=True)

num_epochs = 1
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}")
    for i, (parts, neighbors) in enumerate(train_loader):
        print(f"Batch {i+1}")
        optimizer.zero_grad()
        parts = parts.to(device)
        neighbors = neighbors.to(device)
        output = model(parts)
        loss = criterion(output, neighbors)
        loss.backward()
        optimizer.step()

# Extract the embeddings from the model

embeddings = model.fc1.weight.data
print(len(embeddings))
print(embeddings)



Using mps device
EmbeddingNetwork(
  (fc1): Linear(in_features=1079, out_features=10, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=10, out_features=1079, bias=True)
)
Epoch 1
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19
Batch 20
Batch 21
Batch 22
Batch 23
Batch 24
Batch 25
Batch 26
Batch 27
Batch 28
Batch 29
Batch 30
Batch 31
Batch 32
Batch 33
Batch 34
Batch 35
Batch 36
Batch 37
Batch 38
Batch 39
Batch 40
Batch 41
Batch 42
Batch 43
Batch 44
Batch 45
Batch 46
Batch 47
Batch 48
Batch 49
Batch 50
Batch 51
Batch 52
Batch 53
Batch 54
Batch 55
Batch 56
Batch 57
Batch 58
Batch 59
Batch 60
Batch 61
Batch 62
Batch 63
Batch 64
Batch 65
Batch 66
Batch 67
Batch 68
Batch 69
Batch 70
Batch 71
Batch 72
Batch 73
Batch 74
Batch 75
Batch 76
Batch 77
Batch 78
Batch 79
Batch 80
Batch 81
Batch 82
Batch 83
Batch 84
Batch 85
Batch 86
Batch 87
Batch 88
Batch 89
Batch 90
Batch 91
Batc

In [ ]:
# check equivalence implementation
part1 = Part(202, 2)
part2 = Part(202, 2)
part3 = Part(202, 3)
print(f"Part1: id={id(part1)}, hash={hash(part1)}")
print(f"Part2: id={id(part2)}, hash={hash(part2)}")

print(part1.equivalent(part2))

print(part1.get_part_id() == part2.get_part_id())  
print(part1.get_family_id() == part2.get_family_id()) 
print(Part(202, 2) == Part(202, 2)) 
print(Part(202, 2).__eq__(Part(202, 2)))#
print(part1 == part2)
print(part1 == part3)